__Compare TE FDFD solution with analytical solution__

Daniel Köhn 
Kiel, 01/09/2017

__Import Libraries__

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
from matplotlib.colors import LightSource, Normalize
from matplotlib.pyplot import gca
from pylab import rcParams
from matplotlib import rc
from copy import copy
import matplotlib.ticker as mtick
import scipy.special as sp
from scipy import signal
from mpl_toolkits.axes_grid1 import make_axes_locatable
import pickle

__Define fonts__

In [ ]:
FSize = 15
font = {'color':  'black',
        'weight': 'normal',
        'size': FSize,
        'family': 'serif'} 
rcParams['figure.figsize'] = 10, 10

__Define Acquisition geometry__

Define source positions

In [ ]:
XSRC = 2.0 + 1e-2
YSRC = 2.0 + 1e-2

Read receiver positions

In [ ]:
rec_pos = np.genfromtxt("../receiver/receiver_2_cross_TE.dat", dtype=None)

XREC = rec_pos[:,0]
YREC = rec_pos[:,1]

__Read or define source wavelet__

In [ ]:
# READ_STF = 0 - use spike wavelet
# READ_STF = 1 - read external wavelet from ASCII file
# READ_STF = 2 - define source wavelet in Jupyter notebook
READ_STF = 2

dt_stf = 1.5e-10   # temporal sampling rate [s]
Tmax = 150e-9      # maximum recording time [s]
fc = 100e6         # Ricker wavelet centre frequency [Hz]

__Calculate source-receiver distance r__

In [ ]:
r = np.sqrt((XREC-XSRC)**2+(YREC-YSRC)**2)
ntr = np.size(r,0)

**Analytical solution for homogeneous acoustic full space at each receiver position**

In [ ]:
def analytical_solution(w,mu0,eps,sigma,r):

    # define effective permittivity
    epse = eps + (((0+1j)*sigma) / w)

    k = w * np.sqrt(mu0 * epse)        # complex wavenumber

    # analytical solution
    p_analy = -w * mu0 * sp.hankel1(0,(r*k)) / 4.0
    
    FD_real = np.real(p_analy)
    FD_imag = np.imag(p_analy)
    
    return FD_real, FD_imag

__Calculate analytical solution for discrete frequencies__

In [ ]:
# define material parameters 
mu0 = 4 * np.pi * 1e-7             # magnetic permeability
eps = 3.54e-11                     # permittivity
sigma = 3e-3                       # conductivity

# define frequencies
FC_low = 1e6
FC_high = 200e6

nf = 400
df = (FC_high - FC_low) / (nf-1)

# TD parameters
TmaxTD = 150e-9       # maximum time TD
TmaxFD = 0.25 / df     # maximum time FD
dt = TmaxFD / nf      # time sampling FD

# maximum time sample of FD2TD corresponding to TmaxTD
nmaxFD = np.int(TmaxTD / dt)

FDFD_real  = np.zeros((nf, ntr))
FDFD_imag  = np.zeros((nf, ntr))

f = FC_low
for i in range (nf):

    w = 2.0 * np.pi * f
    tmp_real, tmp_imag = analytical_solution(w,mu0,eps,sigma,r)
    
    for j in range (ntr):
            FDFD_real[i,j] = tmp_real[j]
            FDFD_imag[i,j] = tmp_imag[j]
    f +=df

In [ ]:
# Read source wavelet
if(READ_STF==1):
    
    wavelet = np.genfromtxt("../TD_seismograms/source.txt", dtype=None)
    nt_stf = np.size(wavelet,0)
    dt_stf = 0.1e-9

if(READ_STF==2):        
    
    nt_stf = np.int(Tmax / dt_stf)
    
t_stf = np.linspace(0, dt_stf*nt_stf, nt_stf, endpoint=True)

# define Ricker wavelet
if(READ_STF==2):        
    
    ts = 1.0 / fc
    tau = np.pi * (t_stf - 1.5 * ts) / (1.5 * ts)
    wavelet = (1.0 - 4.0 * tau * tau) * np.exp(-2.0 * tau * tau)
    
# plot wavelet
#if(READ_STF):
    #plt.plot(t_stf*1e9,wavelet)
    #plt.show()

In [ ]:
if(READ_STF):

    # Calculate DFT of time-domain Ricker wavelet
    # -------------------------------------------
    stf_real  = np.zeros((nf,1))
    stf_imag  = np.zeros((nf,1))

    f = FC_low
    for i in range (nf):

        for j in range (nt_stf):
            
            stf_real[i] += wavelet[j] * np.cos(2.0 * t_stf[j] * f * np.pi) * dt_stf;
            stf_imag[i] += wavelet[j] * np.sin(2.0 * t_stf[j] * f * np.pi) * dt_stf;
        
        f +=df
    
    FDFD_stf_real = np.zeros((nf, ntr))
    FDFD_stf_imag = np.zeros((nf, ntr))
    
    # assemble complex data
    #window_stf = signal.tukey(nf,2)
    
    #for i in range (nf):
    #    stf_real[i] *= window_stf[i]
    #    stf_imag[i] *= window_stf[i]
    
    # copy STF FD data into 2D matrix
    for i in range (nf):
        for j in range (ntr):
        
            FDFD_stf_real[i,j] = stf_real[i]
            FDFD_stf_imag[i,j] = stf_imag[i]

    # assemble complex data
    FDFD_stf = FDFD_stf_real + 1j*FDFD_stf_imag
    
    #tmp_stf = stf_real + 1j*stf_imag
    #FD_wavelet = np.concatenate((tmp_stf, np.zeros((nf, 1)), np.zeros((nf, 1)), np.flipud(tmp_stf)))
    
    # IFFT source wavelet
    #TD_wavelet = np.fft.ifft(FD_wavelet)
    #TD_wavelet = np.real(TD_wavelet)
    #TD_wavelet_final = TD_wavelet[1:nt_stf]
    
    #plt.imshow(np.real(FDFD_stf),aspect=0.5)
    #plt.show()

In [ ]:
# taper FD data
window = signal.tukey(nf,2)

for i in range (ntr):
    FDFD_real[:,i] *= window
    FDFD_imag[:,i] *= window
    
# assemble complex data
FDFD = FDFD_real + 1j*FDFD_imag
FDFD_real = None
FDFD_imag = None
#FDFD = np.transpose(FDFD)

# convole spike data with source wavelet
tmp = FDFD * FDFD_stf

FDFD = None
FDFD = tmp

tmp1 = np.concatenate((FDFD, np.zeros((nf, ntr)), np.zeros((nf, ntr)), np.flipud(FDFD)))
#tmp1 = np.concatenate((FDFD, np.zeros((nf, ntr))))
FDFD = None
FDFD = tmp1

In [ ]:
#plt.imshow(np.real(FDFD), aspect=0.5)
#plt.show()

__IFFT data__

In [ ]:
# transformation from FD to TD
FD2TD = np.fft.ifft(FDFD,axis=0)
FD2TD = np.real(FD2TD)
tmp1 = None

# extract FD2TD data up to TmaxTD
tmp1 = FD2TD[1:nmaxFD,:]
FD2DTD = None

# clean memory
tmp = None

In [ ]:
clip=2e-2
plt.imshow(tmp1/np.amax(tmp1),cmap='gray',vmin=-clip, vmax=clip, extent = [1,117,np.max(t_stf*1e9),np.min(t_stf*1e9)])
plt.show()

In [ ]:
np.savetxt('analytical_hom.txt', tmp1)